# Produtividade na Agricultura

Os dados utilizados nessa análise vieram da [tabela 6588, do Levantamento Sistemático da Produção Agrícula](https://sidra.ibge.gov.br/tabela/6588) do IBGE.


## Carregamento de bibliotecas

In [1]:
#download e manipulacao dos dados
import pandas as pd
import numpy as np

#graficos
import altair as alt

#painel interativo
import panel as pn

#utilidades
from datetime import datetime

In [2]:
pn.extension()

## Carregamento e tratamento dos dados

In [3]:
# caminho utilizado para download dos dados utilizando a API do IBGE
path_lspa = 'https://sidra.ibge.gov.br/geratabela?format=us.csv&name=tabela6588.csv&terr=N&rank=-&query=t/6588/n1/all/v/all/p/first%20173/c48/all/l/,,p%2Bv%2Bc48%2Bt'

In [4]:
lspa = pd.read_csv(
    path_lspa,
    skiprows=1,
    skipfooter=18,
    engine='python',
    na_values=["...", "-"],
    parse_dates=['Mês'],
    dtype={
        'Variável': str,
        'Produto das lavouras': str,
        'Área': str,
        'Valor': np.float64,
    }
)

In [5]:
lspa.head()

,Mês,Variável,Produto das lavouras,Unnamed: 3,Unnamed: 4
0,setembro 2006,Área plantada (Hectares),Total,Brasil,62570118
1,setembro 2006,Área plantada (Hectares),"1 Cereais, leguminosas e oleaginosas",Brasil,46358224
2,setembro 2006,Área plantada (Hectares),1.1 Algodão herbáceo,Brasil,899590
3,setembro 2006,Área plantada (Hectares),1.2 Amendoim (1ª Safra),Brasil,78568
4,setembro 2006,Área plantada (Hectares),1.3 Amendoim (2ª Safra),Brasil,29595


In [6]:
lspa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31140 entries, 0 to 31139
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Mês                   31140 non-null  object
 1   Variável              31140 non-null  object
 2   Produto das lavouras  31140 non-null  object
 3   Unnamed: 3            31140 non-null  object
 4   Unnamed: 4            28771 non-null  object
dtypes: object(5)
memory usage: 1.2+ MB


Alguns dos dados precisam ser convertidos e transformados para formatos mais úteis.

Abaixo, fazemos essas transformações.

In [7]:
lspa = lspa.rename(columns={'Unnamed: 3':"Unidade Territorial", 'Unnamed: 4':"Valor"
})

In [8]:
FULL_MONTHS = {'janeiro': 1,  'fevereiro': 2, 'março': 3,    'abril': 4,
               'maio': 5,     'junho': 6,     'julho': 7,     'agosto': 8,
               'setembro': 9, 'outubro': 10,  'novembro': 11, 'dezembro': 12}

In [9]:
lspa['Mês'] = lspa['Mês'].str.split(' ').apply(lambda row: datetime(int(row[1]), FULL_MONTHS[row[0]], 1))

In [10]:
lspa['Unidade Territorial'].unique()

array(['Brasil'], dtype=object)

Só solicitamos os dados a nível Nacional, a coluna não agrega nenhuma informação e será dropada

In [11]:
lspa = lspa.drop(columns=['Unidade Territorial'])

Os produtos listados abaixo tiveram uma quebra significativa de metologia no meio do período.

Os dados relativos a esses produtos são descartados.

In [12]:
produtos_dropados = [
    '2 Abacaxi', '3 Alho', '13 Cebola',
    '14 Coco-da-baía', '16 Guaraná', '19 Maçã',
    '20 Malva', '22 Pimenta-do-reino', '23 Sisal ou agave', 'Total'
]

lspa = lspa[~lspa['Produto das lavouras'].isin(produtos_dropados)]

## Preparação do painel interativo

In [13]:
def plot_producao(df, produto):
    dataf = df[
        (df['Variável'] == 'Produção (Toneladas)') &
        (df['Produto das lavouras'] == produto)
    ]

    return alt.Chart(dataf).mark_line().encode(
        x=alt.X("Mês:T", title='Período'),
        y=alt.Y('Valor:Q', title='Toneladas'),
        tooltip=alt.Tooltip(["Mês:T", 'Valor:Q'])
    ).properties(
        title=f"{produto} - Produção"
    )

def plot_areas(df, produto):
    dataf = df[
        (df['Variável'].isin(['Área plantada (Hectares)', 'Área colhida (Hectares)'])) &
        (df['Produto das lavouras'] == produto)
    ]

    return alt.Chart(dataf).mark_line().encode(
        x=alt.X("Mês:T", title='Período'),
        y=alt.Y("Valor:Q", title='Hectares'),
        color=alt.Color("Variável:N"),
        tooltip=alt.Tooltip(["Mês:T", "Valor:Q", "Variável:N"])
    ).properties(
        title=f"{produto} - Áreas"
    )

def plot_rendimento(df, produto):
    dataf = df[
        (df['Variável'] == 'Rendimento médio (Quilogramas por Hectare)') &
        (df['Produto das lavouras'] == produto)
    ]

    return alt.Chart(dataf).mark_line().encode(
        x=alt.X("Mês:T", title="Período"),
        y=alt.Y("Valor:Q", title="Kg/hectare"),
        tooltip=alt.Tooltip(["Mês:T", "Valor:Q"])
    ).properties(
        title=f"{produto} - Rendimento médio"
    )
    

In [14]:
def plot_grafico_produto(tipo_grafico, Produto, dados=lspa):
    tipos = {
        'producao': plot_producao,
        'areas': plot_areas,
        'rendimento': plot_rendimento,
    }
    
    return tipos[tipo_grafico](dados, Produto)

In [15]:
lspa.head()

,Mês,Variável,Produto das lavouras,Valor
1,2006-09-01,Área plantada (Hectares),"1 Cereais, leguminosas e oleaginosas",46358224
2,2006-09-01,Área plantada (Hectares),1.1 Algodão herbáceo,899590
3,2006-09-01,Área plantada (Hectares),1.2 Amendoim (1ª Safra),78568
4,2006-09-01,Área plantada (Hectares),1.3 Amendoim (2ª Safra),29595
5,2006-09-01,Área plantada (Hectares),1.4 Arroz,3001450


In [16]:
panel = pn.interact(
    plot_grafico_produto,
    tipo_grafico=pn.widgets.Select(
        options={
            "Produção":'producao', 
            "Áreas Plantada/Colhida":'areas', 
            "Rendimento médio por Hectare":'rendimento',},
        name="Tipo de Gráfico"
    ),
    Produto=lspa['Produto das lavouras'].unique().tolist()
)

## Painel Interativo

In [17]:
pn.Row(panel[0], panel[1])


pn.extension('vega')



Row
    [0] Column
        [0] Select(name='Tipo de Gráfico', options={'Produção': 'producao', ...}, value='producao')
        [1] Select(name='Produto', options=['1 Cereais, leguminosas e...], value='1 Cereais, leguminosas e...)
    [1] Row
        [0] Vega(Chart, name='interactive00100')